In [10]:
! pip install transformers torch

In [15]:
import pandas as pd

# Load the datasets from the provided paths in Google Colab
train_df = pd.read_csv('/content/Corona_NLP_train.csv', encoding='latin1')
test_df = pd.read_csv('/content/Corona_NLP_test.csv', encoding='latin1')

# Display the first few rows of the training data to check
train_df.head()
# Select a subset of 100 samples from the training dataset
train_df = train_df.iloc[:100] # Use .iloc to select rows by position

In [18]:
from transformers import pipeline

# To load the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

# To test the model with multiple reviews from the subset
results = []
for i in range(10):  # Testing the first 10 reviews
    review = train_df.iloc[i]["OriginalTweet"] # Access the 'OriginalTweet' column using .iloc for position-based indexing
    sentiment = classifier(review)
    results.append((review, sentiment))

# To show the results
for i, (review, sentiment) in enumerate(results):
    print(f"Review {i+1}: {review}")
    print(f"Sentiment: {sentiment}")
    print("-" * 40)

Review 1: @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
Sentiment: [{'label': 'NEGATIVE', 'score': 0.9927721619606018}]
----------------------------------------
Review 2: advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order
Sentiment: [{'label': 'NEGATIVE', 'score': 0.9970411658287048}]
----------------------------------------
Review 3: Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P
Sentiment: [{'label': 'NEGATIVE', 'score': 0.9947763681411743}]
----------------------------------------





#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j
Sentiment: [{'label': 'NEGATIVE', 'score': 0.99

In [22]:
from transformers import BertTokenizer, BertModel

# Load the specified BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')




In [35]:
import torch

# Get embeddings from BERT
def get_bert_embeddings(tokens):
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Tokenize the training data (assuming 'train_df' contains your training data)
X_train_texts = train_df['OriginalTweet'].tolist() # Extract the text column
X_train_tokens = tokenizer(X_train_texts, truncation=True, padding=True, return_tensors='pt') # Tokenize

X_train_bert = get_bert_embeddings(X_train_tokens)

In [37]:
!pip install gensim
!pip install sklearn


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [38]:
# Preprocess the data: only keep the text and sentiment labels
train_df = train_df[['OriginalTweet', 'Sentiment']] # Keep only relevant columns

# Convert sentiment labels to binary for simplicity (positive/negative)
train_df['Sentiment'] = train_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0)


<ipython-input-38-299795eb789d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Sentiment'] = train_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0)


In [42]:
# Download and load pre-trained GloVe embeddings (e.g., GloVe 50d)
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

glove_model = {}
with open("glove.6B.50d.txt", "r", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        glove_model[word] = vector

def get_glove_embeddings(texts):
    embeddings = []
    for text in texts:
        words = text.split()
        word_vectors = [glove_model[word] for word in words if word in glove_model]
        if word_vectors:
            embeddings.append(np.mean(word_vectors, axis=0))
        else:
            embeddings.append(np.zeros(50))  # Adjust for 50 dimensions
    return np.array(embeddings)

X_train_glove = get_glove_embeddings(train_df['OriginalTweet'])


--2024-08-31 22:01:21--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-31 22:01:21--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-31 22:01:21--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

NameError: name 'np' is not defined

In [ ]:
import numpy as np

glove_model = {}
with open("glove.6B.50d.txt", "r", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")  # Now np is defined
        glove_model[word] = vector


!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

glove_model = {}
with open("glove.6B.50d.txt", "r", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.array(values[1:], dtype="float32")
        glove_model[word] = vector

def get_glove_embeddings(texts):
    embeddings = []
    for text in texts:
        words = text.split()
        word_vectors = [glove_model[word] for word in words if word in glove_model]
        if word_vectors:
            embeddings.append(np.mean(word_vectors, axis=0))
        else:
            embeddings.append(np.zeros(50))  # Adjust for 50 dimensions
    return np.array(embeddings)

X_train_glove = get_glove_embeddings(train_df['OriginalTweet'])


--2024-08-31 22:14:03--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-31 22:14:03--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-31 22:14:04--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split the data
X_train, X_val, y_train, y_val = train_test_split(X_train_bert, train_df['Sentiment'], test_size=0.2, random_state=42)

# Train and evaluate models
def evaluate_embeddings(X_train, X_val, y_train, y_val):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    return accuracy, precision, recall, f1

# Evaluate BERT embeddings
bert_metrics = evaluate_embeddings(X_train_bert, X_train_bert, y_train, y_val)
print(f"BERT - Accuracy: {bert_metrics[0]}, Precision: {bert_metrics[1]}, Recall: {bert_metrics[2]}, F1-Score: {bert_metrics[3]}")

# Evaluate Word2Vec embeddings
X_train, X_val, y_train, y_val = train_test_split(X_train_word2vec, train_df['Sentiment'], test_size=0.2, random_state=42)
word2vec_metrics = evaluate_embeddings(X_train, X_val, y_train, y_val)
print(f"Word2Vec - Accuracy: {word2vec_metrics[0]}, Precision: {word2vec_metrics[1]}, Recall: {word2vec_metrics[2]}, F1-Score: {word2vec_metrics[3]}")

# Evaluate GloVe embeddings
X_train, X_val, y_train, y_val = train_test_split(X_train_glove, train_df['Sentiment'], test_size=0.2, random_state=42)
glove_metrics = evaluate_embeddings(X_train, X_val, y_train, y_val)
print(f"GloVe - Accuracy: {glove_metrics[0]}, Precision: {glove_metrics[1]}, Recall: {glove_metrics[2]}, F1-Score: {glove_metrics[3]}")
